In [ ]:
# External libraries
import numpy as np
from matplotlib import pyplot as plt

# Drake dependencies
from pydrake.all import (
    DiagramBuilder,
    Simulator,
    StartMeshcat,
    MultibodyPlant,
    Demultiplexer,
    DiscreteContactApproximation,
    ConstantVectorSource,
    Parser,
    AddMultibodyPlantSceneGraph,
    ConstantVectorSource,
    DiagramBuilder,
    JointSliders,
    MeshcatVisualizer,
    MeshcatVisualizerParams,
    MultibodyPlant,
    MultibodyPositionToGeometryPose,
    Multiplexer,
    Parser,
    PrismaticJoint,
    SceneGraph,
    SpatialInertia,
    Sphere,
    UnitInertia,
    MeshcatVisualizerParams,
    LoadModelDirectivesFromString,
    RigidTransform,
    ProcessModelDirectives,
    AddDefaultVisualization,
    SimIiwaDriver,
    IiwaControlMode,
    PortSwitch
)

# Custom classes and functions
from ShishKebot.CartesianStiffnessController import CartesianStiffnessController
from ShishKebot.TrajectoryPublisher import TrajectoryPublisher
from ShishKebot.StateMachine import StateMachine
from ShishKebot.Perception import AddRgbdSensors

# Helper functions
import manipulation
from manipulation.meshcat_utils import MeshcatSliders, StopButton
from manipulation.scenarios import AddShape
from manipulation.utils import RenderDiagram, ConfigureParser
import os

In [ ]:
meshcat = StartMeshcat()
meshcat.SetProperty("/Background", "visible", False)

In [ ]:
iiwa1_directive = f"""
directives:
- add_model:
    name: iiwa1
    file: package://drake_models/iiwa_description/sdf/iiwa7_with_box_collision.sdf
    default_joint_positions:
        iiwa_joint_1: [0.0]
        iiwa_joint_2: [0.6]
        iiwa_joint_3: [0.0]
        iiwa_joint_4: [-1.75]
        iiwa_joint_5: [0.0]
        iiwa_joint_6: [1.0]
        iiwa_joint_7: [0.0]
- add_weld:
    parent: world
    child: iiwa1::iiwa_link_0
    X_PC:
        translation: [0, 0.5, 0]
- add_model:
    name: wsg1
    file: package://manipulation/schunk_wsg_50_welded_fingers.sdf
- add_weld:
    parent: iiwa1::iiwa_link_7
    child: wsg1::body
    X_PC:
        translation: [0, 0, 0.09]
        rotation: !Rpy {{ deg: [90, 0, 90]}}

- add_model:
    name: skewer
    file: file://{os.getcwd()}/Models/skewer_5mm.sdf
- add_weld:
    parent: wsg1::body
    child: skewer::skewer_5mm
    X_PC:
        translation: [0, -0.2, 0]
        rotation: !Rpy {{ deg: [270, 0, 0]}}
"""

iiwa2_directive = f"""
directives:
- add_model:
    name: iiwa2
    file: package://drake_models/iiwa_description/sdf/iiwa7_with_box_collision.sdf
    default_joint_positions:
        iiwa_joint_1: [0.0]
        iiwa_joint_2: [0.6]
        iiwa_joint_3: [0.0]
        iiwa_joint_4: [-1.75]
        iiwa_joint_5: [0.0]
        iiwa_joint_6: [1.0]
        iiwa_joint_7: [0.0]
- add_weld:
    parent: world
    child: iiwa2::iiwa_link_0
    X_PC:
        translation: [0, -0.5, 0]
- add_model:
    name: wsg2
    file: package://manipulation/schunk_wsg_50_welded_fingers.sdf
- add_weld:
    parent: iiwa2::iiwa_link_7
    child: wsg2::body
    X_PC:
        translation: [0, 0, 0.09]
        rotation: !Rpy {{ deg: [90, 0, 90]}}
"""

world_directive = f"""
directives:
- add_model:
    name: table
    file: file://{os.getcwd()}/Models/ground.sdf
- add_weld:
    parent: world
    child: table::base

- add_model:
    name: cube
    file: file://{os.getcwd()}/Models/cube_food.sdf
    default_free_body_pose:
        cube_food:
            rotation: !Rpy {{ deg: [{np.random.rand()*180}, {np.random.rand()*180}, {np.random.rand()*180}]}}
            translation: {[-0.4-np.random.rand()*0.6, np.random.rand() - 1, 0]}
- add_model:
    name: cube2
    file: file://{os.getcwd()}/Models/cube_food.sdf
    default_free_body_pose:
        cube_food:
            rotation: !Rpy {{ deg: [{np.random.rand()*180}, {np.random.rand()*180}, {np.random.rand()*180}]}}
            translation: {[-0.4-np.random.rand()*0.6, np.random.rand() - 1, .1]}
- add_model:
    name: cube3
    file: file://{os.getcwd()}/Models/cube_food.sdf
    default_free_body_pose:
        cube_food:
            rotation: !Rpy {{ deg: [{np.random.rand()*180}, {np.random.rand()*180}, {np.random.rand()*180}]}}
            translation: {[-0.4-np.random.rand()*0.6, np.random.rand() - 1, .2]}

- add_frame:
    name: camera0_origin
    X_PF:
        base_frame: world
        # rotation: !Rpy {{ deg: [180, 0, 0]}}
        # translation: [0, 0, 4]
        rotation: !Rpy {{ deg: [225, 0, 0]}}
        translation: [-0.7, -1.5, 0.5]
- add_model:
    name: camera0
    file: package://manipulation/camera_box.sdf
- add_weld:
    parent: camera0_origin
    child: camera0::base

- add_frame:
    name: camera1_origin
    X_PF:
        base_frame: world
        rotation: !Rpy {{ deg: [135, 0, 0]}}
        translation: [-0.7, 0.5, 0.5]
- add_model:
    name: camera1
    file: package://manipulation/camera_box.sdf
- add_weld:
    parent: camera1_origin
    child: camera1::base
"""

In [ ]:
builder = DiagramBuilder()

In [ ]:
# Add iiwa1 to the scene
# iiwa1_plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=1e-3)
# iiwa1_plant.set_name("iiwa1_plant")
# scene_graph.set_name("scene_graph")
iiwa1_plant = MultibodyPlant(time_step=1e-3)
iiwa1_plant.set_discrete_contact_approximation(DiscreteContactApproximation.kSap)
parser = Parser(iiwa1_plant)
ConfigureParser(parser)
directives = LoadModelDirectivesFromString(iiwa1_directive)
models = ProcessModelDirectives(directives, iiwa1_plant, parser)
# iiwa1 = iiwa1_plant.GetModelInstanceByName("iiwa1")
# wsg1 = iiwa1_plant.GetModelInstanceByName("wsg1")
iiwa1_plant.Finalize()

# Meshcat
# params = MeshcatVisualizerParams()
# params.prefix = "iiwa1"
# MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat, params)

In [ ]:
# Add iiwa2 to the scene
# iiwa2_plant, iiwa2_scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=1e-3)
# iiwa2_plant.set_name("iiwa2_plant")
# iiwa2_scene_graph.set_name("iiwa2_scene_graph")
iiwa2_plant = MultibodyPlant(time_step=1e-3)
iiwa2_plant.set_discrete_contact_approximation(DiscreteContactApproximation.kSap)
parser = Parser(iiwa2_plant)
ConfigureParser(parser)
directives = LoadModelDirectivesFromString(iiwa2_directive)
models = ProcessModelDirectives(directives, iiwa2_plant, parser)
# iiwa2 = iiwa2_plant.GetModelInstanceByName("iiwa2")
# wsg2 = iiwa2_plant.GetModelInstanceByName("wsg2")
iiwa2_plant.Finalize()

# Meshcat
# params = MeshcatVisualizerParams()
# params.prefix = "iiwa2"
# MeshcatVisualizer.AddToBuilder(builder, iiwa2_scene_graph, meshcat, params)

In [ ]:
# Add world models to the scene
world_plant, world_scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=1e-3)
world_plant.set_name("world_plant")
world_scene_graph.set_name("world_scene_graph")
world_plant.set_discrete_contact_approximation(DiscreteContactApproximation.kSap)
parser = Parser(world_plant)
ConfigureParser(parser)
directives = LoadModelDirectivesFromString(world_directive)
models = ProcessModelDirectives(directives, world_plant, parser)
directives = LoadModelDirectivesFromString(iiwa1_directive)
models = ProcessModelDirectives(directives, iiwa1_plant, parser)
iiwa1 = iiwa1_plant.GetModelInstanceByName("iiwa1")
wsg1 = iiwa1_plant.GetModelInstanceByName("wsg1")
directives = LoadModelDirectivesFromString(iiwa2_directive)
models = ProcessModelDirectives(directives, iiwa2_plant, parser)
iiwa2 = iiwa2_plant.GetModelInstanceByName("iiwa2")
wsg2 = iiwa2_plant.GetModelInstanceByName("wsg2")
world_plant.Finalize()

# Meshcat
params = MeshcatVisualizerParams()
params.prefix = "world"
MeshcatVisualizer.AddToBuilder(builder, world_scene_graph, meshcat, params)

# Add the cameras to the diagram
pcd_sensors = AddRgbdSensors(builder, world_plant, world_scene_graph)

In [ ]:
# Sim controller 1
sim_iiwa_controller_1 = builder.AddNamedSystem("iiwa1_sim_driver", SimIiwaDriver(
  control_mode=IiwaControlMode.kPositionAndTorque, 
  controller_plant=iiwa1_plant, 
  ext_joint_filter_tau=0.01,
  kp_gains=np.full(iiwa1_plant.num_positions(iiwa1), 100))
)

# INPUTS
builder.Connect(world_plant.GetOutputPort("iiwa1_state"), sim_iiwa_controller_1.GetInputPort("state"))
builder.Connect(world_plant.GetOutputPort("iiwa1_generalized_contact_forces"), sim_iiwa_controller_1.GetInputPort("generalized_contact_forces"))
# torque - connected to switch_torque
# position - connected to switch_position
# OUTPUTS
builder.Connect(sim_iiwa_controller_1.GetOutputPort("actuation"), world_plant.GetInputPort("iiwa1_actuation"))

In [ ]:
# Joint space trajectory publisher 1
iiwa1_joint_trajectory = builder.AddNamedSystem("iiwa1_joint_trajectory", TrajectoryPublisher(
    plant=iiwa1_plant,
    iiwa_name="iiwa1",
    end_effector_name="wsg1"
))

# INPUTS
builder.Connect(sim_iiwa_controller_1.GetOutputPort("position_measured"), iiwa1_joint_trajectory.GetInputPort("iiwa_position_measured"))
# pose_desired - connected to state machine
# OUTPUTS
# iiwa_position_cmd - connected to switch_position

In [ ]:
# Stiffness controller
stiffness_controller = builder.AddNamedSystem("iiwa1_stiffness_controller", CartesianStiffnessController(
    plant=iiwa1_plant, 
    iiwa_name="iiwa1", 
    end_effector_name="wsg1"))
stiffness_controller.SetGains(
    position=(10.0, 5.0),
    orientation=(10.0, 5.0),
    null_space=5.0
)

# INPUTS
builder.Connect(sim_iiwa_controller_1.GetOutputPort("position_measured"), stiffness_controller.GetInputPort("iiwa_position_measured"))
builder.Connect(sim_iiwa_controller_1.GetOutputPort("velocity_estimated"), stiffness_controller.GetInputPort("iiwa_velocity_measured"))
# pose_desired - connected to iiwa1_pose_trajectory
# OUTPUTS
# iiwa_torque_cmd - connected to switch_torque
# iiwa_position_cmd - connected to switch_position

In [ ]:
# Pose space trajectory publisher
iiwa1_pose_trajectory = builder.AddNamedSystem("iiwa1_pose_trajectory", TrajectoryPublisher(
    plant=iiwa1_plant,
    iiwa_name="iiwa1",
    end_effector_name="wsg1",
    pose_speed=1.0
))

# INPUTS
builder.Connect(sim_iiwa_controller_1.GetOutputPort("position_measured"), iiwa1_pose_trajectory.GetInputPort("iiwa_position_measured"))
# pose_desired - connected to state_machine
# OUTPUTS
builder.Connect(iiwa1_pose_trajectory.GetOutputPort("iiwa_pose_cmd"), stiffness_controller.GetInputPort("pose_desired"))

In [ ]:
# Sim controller - iiwa2 has torque mode disabled
sim_iiwa_controller_2 = builder.AddNamedSystem("iiwa2_sim_driver", SimIiwaDriver(
  control_mode=IiwaControlMode.kPositionOnly, 
  controller_plant=iiwa2_plant, 
  ext_joint_filter_tau=0.01,
  kp_gains=np.full(iiwa2_plant.num_positions(iiwa2), 100))
)

# INPUTS
builder.Connect(world_plant.GetOutputPort("iiwa2_state"), sim_iiwa_controller_2.GetInputPort("state"))
builder.Connect(world_plant.GetOutputPort("iiwa2_generalized_contact_forces"), sim_iiwa_controller_2.GetInputPort("generalized_contact_forces"))
# position - connected to iiwa2_joint_trajectory
# OUTPUTS
builder.Connect(sim_iiwa_controller_2.GetOutputPort("actuation"), world_plant.GetInputPort("iiwa2_actuation"))

In [ ]:
# Joint space trajectory publisher
iiwa2_joint_trajectory = builder.AddNamedSystem("iiwa2_joint_trajectory", TrajectoryPublisher(
    plant=iiwa2_plant,
    iiwa_name="iiwa2",
    end_effector_name="wsg2"
))

# INPUTS
builder.Connect(sim_iiwa_controller_2.GetOutputPort("position_measured"), iiwa2_joint_trajectory.GetInputPort("iiwa_position_measured"))
# pose_desired - connected to state_machine
# OUTPUTS
builder.Connect(iiwa2_joint_trajectory.GetOutputPort("iiwa_position_cmd"), sim_iiwa_controller_2.GetInputPort("position"))

In [ ]:
state_machine = builder.AddNamedSystem("state_machine", StateMachine(
    iiwa1_plant, 
    iiwa2_plant,
    world_plant,
    num_cameras=2,
    meshcat=meshcat
    ))

# INPUTS - current position of each iiwa, cameras
builder.Connect(sim_iiwa_controller_1.GetOutputPort("position_measured"), state_machine.GetInputPort("iiwa1_position_measured"))
builder.Connect(sim_iiwa_controller_2.GetOutputPort("position_measured"), state_machine.GetInputPort("iiwa2_position_measured"))
for i, sensor in enumerate(pcd_sensors):
    builder.Connect(sensor.GetOutputPort("point_cloud"), state_machine.GetInputPort(f"camera{i}_point_cloud"))
# OUTPUTS - desired pose of each iiwa
builder.Connect(state_machine.GetOutputPort("pose_desired_1"), iiwa1_joint_trajectory.GetInputPort("pose_desired"))
builder.Connect(state_machine.GetOutputPort("pose_desired_1"), iiwa1_pose_trajectory.GetInputPort("pose_desired"))
builder.Connect(state_machine.GetOutputPort("pose_desired_2"), iiwa2_joint_trajectory.GetInputPort("pose_desired"))
##################################
# UNWIRED: Output("close_gripper")
##################################

In [ ]:
# PortSwitch1 - outputs position commands to the skewer iiwa (iiwa1)
switch_position = builder.AddNamedSystem("position_switch", PortSwitch(7))

# INPUTS - position control from both controllers
builder.Connect(iiwa1_joint_trajectory.GetOutputPort("iiwa_position_cmd"), switch_position.DeclareInputPort("diff_ik"))
builder.Connect(stiffness_controller.GetOutputPort("iiwa_position_cmd"), switch_position.DeclareInputPort("stiffness"))
# OUTPUTS
builder.Connect(switch_position.get_output_port(), sim_iiwa_controller_1.GetInputPort("position"))
builder.Connect(state_machine.GetOutputPort("control_mode"), switch_position.get_port_selector_input_port())

In [ ]:
# PortSwitch2 - outputs torque commands to the skewer iiwa (iiwa1)
switch_torque = builder.AddNamedSystem("torque_switch", PortSwitch(7))
zeros = builder.AddSystem(ConstantVectorSource(np.zeros(7)))

# INPUTS - torque control from stiffness controller, or disabled
builder.Connect(zeros.get_output_port(0), switch_torque.DeclareInputPort("diff_ik"))
builder.Connect(stiffness_controller.GetOutputPort("iiwa_torque_cmd"), switch_torque.DeclareInputPort("stiffness"))
# OUTPUTS
builder.Connect(switch_torque.get_output_port(), sim_iiwa_controller_1.GetInputPort("torque"))
builder.Connect(state_machine.GetOutputPort("control_mode"), switch_torque.get_port_selector_input_port())

In [ ]:
diagram = builder.Build()
simulator = Simulator(diagram)

In [ ]:
simulator.set_target_realtime_rate(1.0)
simulator.AdvanceTo(np.inf)